In [89]:
import knime.scripting.io as knio
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [90]:
service = Service(ChromeDriverManager().install())

# Abrir aba Web
driver = webdriver.Chrome(service=service)
# Localizar website e entrar
website = 'https://desaparecidosdobrasil.org/pesquisa_cadastrodesaparecidos.php'
driver.get(website)
# Maximizar aba Web
driver.maximize_window()

In [91]:
slot_element = driver.find_element(By.XPATH, '//select[@name="example_length"]')
slot_element.click()
option = driver.find_element(By.XPATH, '//*[@id="example_length"]/label/select/option[4]')
option.click()

In [92]:
nomes = []
nome_mãe = []
data_desaparecimento = []
local_desaparecimento = []
links = []
nomes_contato = []
data_nascimento = []
BO =[]
descrição = []
teste = ['https://desaparecidosdobrasil.org/pesquisa_exibecadastrodesaparecidos.php?edit_id=59', 'https://desaparecidosdobrasil.org/pesquisa_exibecadastrodesaparecidos.php?edit_id=64']

In [93]:
font_elements = driver.find_elements(By.XPATH, '//font')

for element in font_elements:
    nomes.append(element.text)


tr_elements = driver.find_elements(By.XPATH, '//tr')

for tr in tr_elements:
    th_elements = tr.find_elements(By.TAG_NAME, 'th')
    
    if len(th_elements) > 1:
        nome = th_elements[4].text if len(th_elements) > 1 and th_elements[1].text else 'N/A'
        nome_mãe.append(nome)
        
        data = th_elements[5].text if len(th_elements) > 2 and th_elements[2].text else 'N/A'
        data_desaparecimento.append(data)
        
        cidade = th_elements[6].text if len(th_elements) > 3 and th_elements[3].text else 'N/A'
        local_desaparecimento.append(cidade)

In [94]:
# Encontra todos os elementos tr com as classes "odd" e "even"
tr_elements = driver.find_elements(By.XPATH, '//tr[contains(@class, "odd") or contains(@class, "even")]')

# Itera sobre cada elemento tr
for tr in tr_elements:
    # Dentro de cada tr, encontra a tag <a> e extrai o atributo href
    a_elements = tr.find_elements(By.TAG_NAME, 'a')
    for a in a_elements:
        href = a.get_attribute('href')
        if href:  # Garante que o href não seja None
            links.append(href)

In [95]:
for website in links:
    driver.get(website)

    try:
        nome_contato = driver.find_element(By.ID, 'txtnomecontato').get_attribute('value')
        nomes_contato.append(nome_contato if nome_contato else "N/A")
    except NoSuchElementException:
        nomes_contato.append("N/A")

    try:
        valor_data_nascimento = driver.find_element(By.ID, 'txtdatanasc').get_attribute('value')
        data_nascimento.append(valor_data_nascimento if valor_data_nascimento else "N/A")
    except NoSuchElementException:
        data_nascimento.append("N/A")

    try:
        numero_BO = driver.find_element(By.ID, 'txtboletimocorrencia').get_attribute('value')
        BO.append(numero_BO if numero_BO else "N/A")
    except NoSuchElementException:
        BO.append("N/A")

    try:
        texto_textarea = driver.find_element(By.ID, 'txtinformacoes').get_attribute('value')
        descrição.append(texto_textarea if texto_textarea else "N/A")
    except NoSuchElementException:
        descrição.append("N/A")


In [96]:
nome_mãe.pop(0)
data_desaparecimento.pop(0)
local_desaparecimento.pop(0)

'Local Desaparecimento'

In [101]:
df_list = pd.DataFrame({'Nome': nomes, 'Nome_Responsável': nomes_contato, 'Nome_Mãe': nome_mãe, 'Local_Desaparecimento': local_desaparecimento, 'Data_Desaparecimento': data_desaparecimento, 'Data_Nascimento': data_nascimento, 'Número_Boletim_de_Ocorrencia': BO, 'Descrição': descrição})
df_list.to_csv('Desaparecidos_Brasil.csv', index=False)

In [ ]:
df_list = pd.DataFrame({'Nome': nomes, 'Nome_Responsável': nomes_contato, 'Nome_Mãe': nome_mãe, 'Local_Desaparecimento': local_desaparecimento, 'Data_Desaparecimento': data_desaparecimento, 'Data_Nascimento': data_nascimento, 'Número_Boletim_de_Ocorrencia': BO, 'Descrição': descrição})
knio.output_tables[0] = knio.Table.from_pandas(df_list)